# Building a multi-organ GRN using score results from PG-SCOTT!

Scores generated in `find_genes_peaks.ipynb`. This notebook will serve to build a gene regulatory network, starting with one organ (heart) and expanding.

In [1]:
# import libraries
import pandas as pd
from scipy.io import mmread
import anndata as ad
import json

In [2]:
# load data
sensitivity_df = pd.read_csv('./data/pg_pairs_sensitivity.csv', index_col=[0, 1], header=[0, 1])
specificity_df = pd.read_csv('./data/pg_pairs_specificity.csv', index_col=[0, 1], header=[0, 1])
universality_df = pd.read_csv('./data/pg_pairs_universality.csv', index_col=[0, 1])
atac_adata = ad.read_h5ad('./data/atac.h5ad')
translation_table = pd.read_table('./data/H12INVIVO/translationTable_hg38.csv', sep=' ')
gene_locs = pd.read_csv('./data/gene_locations_hg38.tsv', sep='\t', index_col='Symbol')
tf_peak_overlap = mmread('./data/tf_peak_overlap.mtx')  # This and following obtained from GRaNIE TF_peak_overlap sparse matrix... could be done from scratch using .bed.gz files
tf_motifs = pd.read_csv('./data/tf_names.csv', header=0, names=['idx', 'tf_file'])['tf_file']
peak_locs = pd.read_csv('./data/peak_names.csv', header=0, names=['idx', 'peak_loc'])['peak_loc']

### Heart GRN
Bipartite, directed graph where each node is either a Peak or Gene. For each node, we have the following information. 
- Peak: Condition sensitivity score, Organ specificity score, universality score
- Gene: Condition sensitivity score, Organ specificity score, universality score, whether it codes for a TF

Each edge indicates:
- Gene $\rightarrow$ Peak: Gene codes for a transcription factor that binds to a region within or overlapping with this peak
- Peak $\rightarrow$ Gene: Peak is located within 150kb of Gene

Questions that can be asked:
- What genes share expression patterns with neighboring peak accessibility? How organ-specific are these peaks and genes?
- What TFs bind to a given peak? Is the TF expressed across organs?

In [3]:
gene_to_idx = {
  gene: (gene, peak)
  for gene, peak in sensitivity_df.index.to_frame(index = False).groupby('Gene').aggregate('first')['Peak'].items()
}
peak_to_idx = {
  peak: (gene, peak)
  for peak, gene in sensitivity_df.index.to_frame(index = False).groupby('Peak').aggregate('first')['Gene'].items()
}
organs = sensitivity_df.columns.get_level_values(0).unique()

In [5]:
# Build nodes
tf_names = set(translation_table['SYMBOL'])
gene_to_info = [{ 'id': gene,
                        'node_type': 'gene', 
                        'chr': f'chr{row['Chromosome']}',
                        'start': row['Begin'],
                        'end': row['End'],
                        **{ organ: {
                          'condition_sensitivity': sensitivity_df[organ, 'gene_score'][gene_to_idx[gene]],
                          'organ_specificity': specificity_df[organ, 'gene_score'][gene_to_idx[gene]], 
                        } for organ in organs},
                        'universality': universality_df['gene_score'][gene_to_idx[gene]],
                        'tf_coding': gene in tf_names }
                        for gene, row in gene_locs.loc[sensitivity_df.index.unique(level = 0)].iterrows()]
print(len(gene_to_info))
peak_to_info = [{'id': peak,
                        'node_type': 'peak', 
                        'chr': atac_adata.var['Chr'][f'Interval_{peak}'],
                        'start': atac_adata.var['Start'][f'Interval_{peak}'].item(),
                        'end': atac_adata.var['End'][f'Interval_{peak}'].item(),
                        **{ organ: {
                          'condition_sensitivity': sensitivity_df[organ, 'peak_score'][peak_to_idx[peak]],
                          'organ_specificity': specificity_df[organ, 'peak_score'][peak_to_idx[peak]], 
                        } for organ in organs},
                        'universality': universality_df['peak_score'][peak_to_idx[peak]] } 
                        for peak in sensitivity_df.index.unique(level = 1)]
print(len(peak_to_info))

21119
146156


In [6]:
nodes = [*gene_to_info, *peak_to_info]

In [7]:
# Build edges (going with adjacency list representation for now, though this may complicate the ability to answer some of our questions of interest)

# extract relevant TF information for gene -> peak mappings
motif_to_name = translation_table['SYMBOL'].set_axis(translation_table['HOCOID'])
peak_name_to_loc = atac_adata.var['Chr'].astype(str) + ':' + atac_adata.var['Start'].astype(str) + '-' + atac_adata.var['End'].astype(str)
peak_loc_to_name = atac_adata.var_names.map(lambda peak_name: int(peak_name[9:])).to_series(index = peak_name_to_loc)
peak_idx, tf_idx = tf_peak_overlap.nonzero()
tf_to_peaks = { motif_to_name[tf_motif]: peak_loc_to_name[peak_locs].values.tolist() for tf_motif, peak_locs in peak_locs[peak_idx].groupby(tf_motifs[tf_idx].values) }
len(tf_to_peaks)

940

In [8]:
# extract peak -> gene mappings from scoring dataframe index
peak_to_genes = sensitivity_df.index.get_level_values(0).groupby(sensitivity_df.index.get_level_values(1))
peak_to_genes = { k: v.tolist() for k, v in peak_to_genes.items() }
len(peak_to_genes)

146156

In [9]:
edges = {}
edges.update(tf_to_peaks)
edges.update(peak_to_genes)

In [10]:
with open("./data/grn.json", 'w') as fp:
  json.dump({'nodes': nodes, 'edges': edges}, fp)